In [ ]:
import math
import pandas as pd
import numpy as np
from together import Together
apikey = input()
client = Together(api_key=apikey)


In [2]:
data_long = pd.read_csv('Datasets/Fig-QA Dataset copy.csv')
data_long = data_long[(data_long.labels!=-1)]

data_test = pd.read_excel('Datasets/figqa test labels.xlsx')

data_negsample = pd.read_excel('Datasets/negation sentences.xlsx')


results_folder = "Results Nov25"

In [ ]:
    # prepare midphrase prompt old
    '''
    if sdata.iloc()[i]['startphrase'][-1] == ".":
        prompt1 = "" + (sdata.iloc()[i]['startphrase'] + midphrase + sdata.iloc()[i]['ending1'])
        prompt2 = "" + (sdata.iloc()[i]['startphrase'] + midphrase + sdata.iloc()[i]['ending2'])
    else:
        prompt1 = "" + (sdata.iloc()[i]['startphrase'] + "." + midphrase + sdata.iloc()[i]['ending1'])
        prompt2 = "" + (sdata.iloc()[i]['startphrase'] + "." + midphrase + sdata.iloc()[i]['ending2'])
    '''

In [4]:
# Llama midphrase

#modeln = ("meta-llama/Llama-2-70b-hf", "Llama2-70B")
#modeln = ("meta-llama/Llama-2-7b-chat-hf", "Llama2-7B-chat")
#modeln = ("meta-llama/Llama-2-13b-chat-hf", "Llama2-13B-chat")
modeln = ("meta-llama/Meta-Llama-3-70B-Instruct-Turbo", "Llama3-70B-Instruct")
#modeln = ("meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", "Llama3.1-70B-Instruct")
#modeln = ("meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo", "Llama3.1-405B-Instruct")
#modeln = ("meta-llama/Llama-3.3-70B-Instruct-Turbo", "Llama3.3-70B-Instruct")

test = "M_litneg"
sdata = data_negsample[:]

model = modeln[0]
modelstr = modeln[1]
midphrases = ((" That is to say, ", 0), (" In other words, ", 0), (" Put differently, ",0), (" ", 1), ("\n", 1))


for mid_id in enumerate(midphrases[:]):
    midp = mid_id[1]
    midphrase = midp[0]
    capitalize = midp[1]
    scores1 = []
    scores2 = []
    classifications = []
    id_check = []
    errs = 0


    for i in range(len(sdata[:])):
        #i += 3500
        # prepare prompt

        # add "." if not present
        startphrase = sdata.iloc()[i]['startphrase']
        if startphrase[-1] != ".":
            startphrase = startphrase + "."
        ending1 = sdata.iloc()[i]['ending1']
        if ending1[-1] != ".":
            ending1 = ending1 + "."
        ending2 = sdata.iloc()[i]['ending2']
        if ending2[-1] != ".":
            ending2 = ending2 + "."

        # capitalize
        startphrase = startphrase[0].upper() + startphrase[1:]
        if capitalize == 1:
            ending1 = ending1[0].upper() + ending1[1:]
            ending2 = ending2[0].upper() + ending2[1:]
        else:
            ending1 = ending1[0].lower() + ending1[1:]
            ending2 = ending2[0].lower() + ending2[1:]

        # contruct prompt
        prompt1 = "" + startphrase + midphrase + ending1
        prompt2 = "" + startphrase + midphrase + ending2
        

        # call model api 
        response1 = client.completions.create(
            model=model,
            prompt=prompt1,
            max_tokens=1,
            #temperature=0.6,
            #top_p=0.95,
            #top_k=50,
            #repetition_penalty=1,
            #stop=["<｜end▁of▁sentence｜>"],
            echo=True,
            logprobs=True,
        )
        response2 = client.completions.create(
            model=model,
            prompt=prompt2,
            max_tokens=1,
            #temperature=0.6,
            #top_p=0.95,
            #top_k=50,
            #repetition_penalty=1,
            #stop=["<｜end▁of▁sentence｜>"],
            echo=True,
            logprobs=True,
        )

        #print("--------")
        #print(response1.usage)
        #print(response2.usage)

        # extract logprobs and calculate sentence likelihood
        # exclude leading space (null logprob)
        sentence_loss1 = 0
        prompt_logprobs = response1.prompt[0].logprobs.token_logprobs[1:]
        try: 
            for log in prompt_logprobs:
                sentence_loss1 += log
            sentence_loss1 /= len(prompt_logprobs)
        except TypeError:
            sentence_loss1 = None
            errs += 1

        sentence_loss2 = 0
        prompt_logprobs = response2.prompt[0].logprobs.token_logprobs[1:]
        try: 
            for log in prompt_logprobs:
                sentence_loss2 += log
            sentence_loss2 /= len(prompt_logprobs)
        except TypeError:
            sentence_loss2 = None
            errs += 1

        scores1.append(sentence_loss1)
        scores2.append(sentence_loss2)

        id_check.append(sdata.iloc()[i]["xid"])

        # assign binary score
        try: 
            if sdata.iloc()[i]['labels'] == 0:
                if  sentence_loss1 > sentence_loss2: 
                    classification = 1
                else:
                    classification = 0
            elif sdata.iloc()[i]['labels'] == 1:
                if  sentence_loss2 > sentence_loss1: 
                    classification = 1
                else:
                    classification = 0
            else:
                classification = ""
        except TypeError:
            classification = ""
        classifications.append(classification)

        if i < 10 or i%400 == 0:
            print(i,"/",len(sdata))
        if i < 10:
            print(prompt1)
            print(prompt2)
            
    print("Errors:", errs)
    # fill remaining rows with blanks
    classifications.extend([""] * (len(sdata)-len(classifications)))
    scores1.extend([""] * (len(sdata)-len(scores1)))
    scores2.extend([""] * (len(sdata)-len(scores2)))
    id_check.extend([""] * (len(sdata)-len(id_check)))

    sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check

sdata.to_csv(results_folder+"/test_together_midphrase_{}_".format(modelstr)+test+".csv", index=False)


0 / 256
She is honest. That is to say, she can be trusted.
She is honest. That is to say, she can not be trusted.
1 / 256
She is dishonest. That is to say, she can be trusted.
She is dishonest. That is to say, she can not be trusted.
2 / 256
The student was cooperative. That is to say, he was helpful.
The student was cooperative. That is to say, he was not helpful.
3 / 256
The student was uncooperative. That is to say, he was helpful.
The student was uncooperative. That is to say, he was not helpful.
4 / 256
The man was stylish. That is to say, the man was cool.
The man was stylish. That is to say, the man was not cool.
5 / 256
The man was unfashionable. That is to say, the man was cool.
The man was unfashionable. That is to say, the man was not cool.
6 / 256
The child was devout. That is to say, the child was religious.
The child was devout. That is to say, the child was not religious.
7 / 256
The child was atheistic. That is to say, the child was religious.
The child was atheistic. T

C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{

0 / 256
She is honest. In other words, she can be trusted.
She is honest. In other words, she can not be trusted.
1 / 256
She is dishonest. In other words, she can be trusted.
She is dishonest. In other words, she can not be trusted.
2 / 256
The student was cooperative. In other words, he was helpful.
The student was cooperative. In other words, he was not helpful.
3 / 256
The student was uncooperative. In other words, he was helpful.
The student was uncooperative. In other words, he was not helpful.
4 / 256
The man was stylish. In other words, the man was cool.
The man was stylish. In other words, the man was not cool.
5 / 256
The man was unfashionable. In other words, the man was cool.
The man was unfashionable. In other words, the man was not cool.
6 / 256
The child was devout. In other words, the child was religious.
The child was devout. In other words, the child was not religious.
7 / 256
The child was atheistic. In other words, the child was religious.
The child was atheistic. I

C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{

0 / 256
She is honest. Put differently, she can be trusted.
She is honest. Put differently, she can not be trusted.
1 / 256
She is dishonest. Put differently, she can be trusted.
She is dishonest. Put differently, she can not be trusted.
2 / 256
The student was cooperative. Put differently, he was helpful.
The student was cooperative. Put differently, he was not helpful.
3 / 256
The student was uncooperative. Put differently, he was helpful.
The student was uncooperative. Put differently, he was not helpful.
4 / 256
The man was stylish. Put differently, the man was cool.
The man was stylish. Put differently, the man was not cool.
5 / 256
The man was unfashionable. Put differently, the man was cool.
The man was unfashionable. Put differently, the man was not cool.
6 / 256
The child was devout. Put differently, the child was religious.
The child was devout. Put differently, the child was not religious.
7 / 256
The child was atheistic. Put differently, the child was religious.
The child w

C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{

0 / 256
She is honest. She can be trusted.
She is honest. She can not be trusted.
1 / 256
She is dishonest. She can be trusted.
She is dishonest. She can not be trusted.
2 / 256
The student was cooperative. He was helpful.
The student was cooperative. He was not helpful.
3 / 256
The student was uncooperative. He was helpful.
The student was uncooperative. He was not helpful.
4 / 256
The man was stylish. The man was cool.
The man was stylish. The man was not cool.
5 / 256
The man was unfashionable. The man was cool.
The man was unfashionable. The man was not cool.
6 / 256
The child was devout. The child was religious.
The child was devout. The child was not religious.
7 / 256
The child was atheistic. The child was religious.
The child was atheistic. The child was not religious.
8 / 256
The hangover made her feel terrible. She felt bad.
The hangover made her feel terrible. She did not feel bad.
9 / 256
The hangover made her feel good. She felt bad.
The hangover made her feel good. She di

C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{

0 / 256
She is honest.
She can be trusted.
She is honest.
She can not be trusted.
1 / 256
She is dishonest.
She can be trusted.
She is dishonest.
She can not be trusted.
2 / 256
The student was cooperative.
He was helpful.
The student was cooperative.
He was not helpful.
3 / 256
The student was uncooperative.
He was helpful.
The student was uncooperative.
He was not helpful.
4 / 256
The man was stylish.
The man was cool.
The man was stylish.
The man was not cool.
5 / 256
The man was unfashionable.
The man was cool.
The man was unfashionable.
The man was not cool.
6 / 256
The child was devout.
The child was religious.
The child was devout.
The child was not religious.
7 / 256
The child was atheistic.
The child was religious.
The child was atheistic.
The child was not religious.
8 / 256
The hangover made her feel terrible.
She felt bad.
The hangover made her feel terrible.
She did not feel bad.
9 / 256
The hangover made her feel good.
She felt bad.
The hangover made her feel good.
She di

C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} 2".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} bin".format(modelstr+" M"+str(mid_id[0]+1))], sdata["{} id_check".format(modelstr+" M"+str(mid_id[0]+1))] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_61272\2473665250.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{

In [7]:
print(i)
print(prompt1)
print(prompt_logprobs)

3870
That game was a squeaker. That is to say, that game was very close.
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [9]:
# Llama logprobs - question prompt - id based mixedprompt

#modeln = ("meta-llama/Llama-2-70b-hf", "Llama2-70B")
#modeln = ("meta-llama/Llama-2-7b-chat-hf", "Llama2-7B-chat")
#modeln = ("meta-llama/Llama-2-13b-chat-hf", "Llama2-13B-chat")
#modeln = ("meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", "Llama3.1-70B-Instruct")
modeln = ("meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo", "Llama3.1-405B-Instruct")
#modeln = ("meta-llama/Llama-3.3-70B-Instruct-Turbo", "Llama3.3-70B-Instruct")


model = modeln[0]
modelstr = modeln[1]

#test = "Q_mixedprompt"
#sdata = data_long[:]

test = "Q_mixedprompt_test"
sdata = data_test[:]

#test = "litneg_question_mixedprompt_fixed"
#sdata = data_negsample[:]


scores1 = []
scores2 = []
classifications = []
id_check = []
prompt_reversed = []
prev_qid = ""
flip_id = 0
for i in range(len(sdata[:])):
    # prepare prompt
    if sdata.iloc()[i]["qid"] != prev_qid:
        flip_id += 1
    flipped = 0 if flip_id%2 == 0 else 1
    if flipped == 0:
        prompt1 = "" + 'Which of the following sentences, sentence A or sentence B, most accurately describes the meaning of the following: "' + sdata.iloc()[i]['startphrase'] + '"? Sentence A: "' + sdata.iloc()[i]['ending1'] + '" Sentence B: "' + sdata.iloc()[i]['ending2'] + '" \nThe answer is sentence A'    
        prompt2 = "" + 'Which of the following sentences, sentence A or sentence B, most accurately describes the meaning of the following: "' + sdata.iloc()[i]['startphrase'] + '"? Sentence A: "' + sdata.iloc()[i]['ending1'] + '" Sentence B: "' + sdata.iloc()[i]['ending2'] + '" \nThe answer is sentence B'    
        prompt_reversed.append(0)
    else:
        prompt1 = "" + 'Which of the following sentences, sentence A or sentence B, most accurately describes the meaning of the following: "' + sdata.iloc()[i]['startphrase'] + '"? Sentence A: "' + sdata.iloc()[i]['ending2'] + '" Sentence B: "' + sdata.iloc()[i]['ending1'] + '" \nThe answer is sentence A'    
        prompt2 = "" + 'Which of the following sentences, sentence A or sentence B, most accurately describes the meaning of the following: "' + sdata.iloc()[i]['startphrase'] + '"? Sentence A: "' + sdata.iloc()[i]['ending2'] + '" Sentence B: "' + sdata.iloc()[i]['ending1'] + '" \nThe answer is sentence B'    
        prompt_reversed.append(1)
       
    # call model api 
    response1 = client.completions.create(
        model=model,
        prompt=prompt1,
        max_tokens=1,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["</s>"],
        echo=True,
        logprobs=True,
    )
    response2 = client.completions.create(
        model=model,
        prompt=prompt2,
        max_tokens=1,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["</s>"],
        echo=True,
        logprobs=True,
    )

    #print("--------")
    #print(response1.usage)
    #print(response2.usage)

    # extract logprobs and calculate sentence likelihood
    # exclude leading space (null logprob)
    sentence_loss1 = 0
    prompt_logprobs = response1.prompt[0].logprobs.token_logprobs[1:]
    for log in prompt_logprobs:
        sentence_loss1 += log
    sentence_loss1 /= len(prompt_logprobs)
    
    sentence_loss2 = 0
    prompt_logprobs = response2.prompt[0].logprobs.token_logprobs[1:]
    for log in prompt_logprobs:
        sentence_loss2 += log
    sentence_loss2 /= len(prompt_logprobs)
    
    scores1.append(sentence_loss1)
    scores2.append(sentence_loss2)
    
    id_check.append(sdata.iloc()[i]["xid"])

    # assign binary score
    if sdata.iloc()[i]['labels'] == 0:
        if  sentence_loss1 > sentence_loss2: 
            classification = 0 if flipped == 1 else 1
        else:
            classification = 1 if flipped == 1 else 0
    elif sdata.iloc()[i]['labels'] == 1:
        if  sentence_loss2 > sentence_loss1: 
            classification = 0 if flipped == 1 else 1
        else:
            classification = 1 if flipped == 1 else 0
    else:
        classification = ""
    classifications.append(classification)
    
    prev_qid = sdata.iloc()[i]["qid"]
    
    if i < 10 or i%250 == 0:
        print(i,"/",len(sdata))
        
#fill remaining rows with blanks
classifications.extend([""] * (len(sdata)-len(classifications)))
scores1.extend([""] * (len(sdata)-len(scores1)))
scores2.extend([""] * (len(sdata)-len(scores2)))
id_check.extend([""] * (len(sdata)-len(id_check)))

sdata["1 {}".format(modelstr)], sdata["2 {}".format(modelstr)], sdata["bin {}".format(modelstr)], sdata["id_check {}".format(modelstr)], sdata["prompt_reversed {}".format(modelstr)] = scores1, scores2, classifications, id_check, prompt_reversed
#print(np.mean(classifications))
sdata.to_csv(results_folder+"/test_together_{}_".format(modelstr)+test+".csv", index=False)


0 / 1146
1 / 1146
2 / 1146
3 / 1146
4 / 1146
5 / 1146
6 / 1146
7 / 1146
8 / 1146
9 / 1146
250 / 1146
500 / 1146
750 / 1146
1000 / 1146


C:\Users\jo16726\AppData\Local\Temp\ipykernel_24040\4260338080.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["1 {}".format(modelstr)], sdata["2 {}".format(modelstr)], sdata["bin {}".format(modelstr)], sdata["id_check {}".format(modelstr)], sdata["prompt_reversed {}".format(modelstr)] = scores1, scores2, classifications, id_check, prompt_reversed
C:\Users\jo16726\AppData\Local\Temp\ipykernel_24040\4260338080.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["1 {}".format(modelstr)], s

In [4]:
# Llama chat output
model = "meta-llama/Llama-2-7b-chat-hf"
modelstr = "Llama2-7b-chat"
#meta-llama/Llama-2-13b-chat-hf
#meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
#meta-llama/Meta-Llama-3-70B-Instruct-Turbo
test = "all-chat-0.6"
sdata = data_long[:20]
midphrases = (" That is to say, ", " In other words, ")
midphrase = midphrases[1]

outs = []
classifications = []
id_check = []
for i in range(len(sdata[:])):
    #i += 3500
    # prepare prompt
    prompt = "" + '" Answer only "A" or "B": ' + 'Which of the following sentences, sentence A or sentence B, most accurately describes the meaning of the following: "' + sdata.iloc()[i]['startphrase'] + '"? Sentence A: "' + sdata.iloc()[i]['ending1'] + '" Sentence B: "' + sdata.iloc()[i]['ending2']
    #print(prompt)
    # call model api 
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user",
            "content": prompt
            }],
        #max_tokens=1,
        max_tokens=10,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["[/INST]","</s>"],
        )

    #print("--------")
    #print(response1.usage)
    #print(response2.usage)

    # extract logprobs and calculate sentence likelihood
    # exclude leading space (null logprob)
    output = response.choices[0].message.content
    outs.append(output)
    
    # assign binary score
    if sdata.iloc()[i]['labels'] == 0:
        if output == 'A': 
            classification = 1
        else:
            classification = 0
    elif sdata.iloc()[i]['labels'] == 1:
        if output == 'B': 
            classification = 1
        else:
            classification = 0
    else:
        classification = ""
    classifications.append(classification)
    
    id_check.append(sdata.iloc()[i]["xid"])

    # assign binary score
    
# fill remaining rows with blanks
classifications.extend([""] * (len(sdata)-len(classifications)))
outs.extend([""] * (len(sdata)-len(outs)))
id_check.extend([""] * (len(sdata)-len(id_check)))

sdata["out {}".format(modelstr)], sdata["bin {}".format(modelstr)], sdata["id_check {}".format(modelstr)] = outs, classifications, id_check
#print(np.mean(classifications))
sdata.to_csv("FIG-QA/test_openai_{}_".format(modelstr)+test+".csv", index=False)

C:\Users\jo16726\AppData\Local\Temp\ipykernel_9172\1328494298.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} out".format(modelstr)], sdata["{} bin".format(modelstr)], sdata["{} id_check".format(modelstr)] = outs, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_9172\1328494298.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} out".format(modelstr)], sdata["{} bin".format(modelstr)], sdata["{} id_check".format(modelstr)] = outs, classifications, id_check
C:

In [12]:
# fill remaining rows with blanks if run partially
classifications.extend([""] * (len(sdata)-len(classifications)))
scores1.extend([""] * (len(sdata)-len(scores1)))
scores2.extend([""] * (len(sdata)-len(scores2)))
id_check.extend([""] * (len(sdata)-len(id_check)))

# save
sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)], sdata["{} id_check".format(modelstr)] = scores1, scores2, classifications, id_check
#print(np.mean(classifications))
sdata.to_csv("FIG-QA/test_together_logprobs_{}_".format(modelstr)+test+".csv", index=False)

C:\Users\jo16726\AppData\Local\Temp\ipykernel_21184\2743726068.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)], sdata["{} id_check".format(modelstr)] = scores1, scores2, classifications, id_check
C:\Users\jo16726\AppData\Local\Temp\ipykernel_21184\2743726068.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)], sd